# 1 数据探索

# 2 特征工程

# 3 模型融合

## voting
Voting（投票机制）分为硬投票和软投票两种，其原理采用少数服从多数的思想，此方法可用于分类问题。

### 软投票示例

**导入相关包**

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.data import iris_data
from mlxtend.plotting import plot_decision_regions


**EnsembleVoteClassifier实现**

In [ ]:
if __name__ == '__main__':
    # Initializing Classifiers
    clf1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='auto')
    clf2 = RandomForestClassifier(random_state=0, n_estimators=100)
    clf3 = SVC(random_state=0, probability=True, gamma='auto')
    eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[2, 1, 1], voting='soft')

    # Loading some example data
    X, y = iris_data()
    X = X[:, [0, 2]]

    # plotting decision regions
    gs = gridspec.GridSpec(1, 4)
    fig = plt.figure(figsize=(16, 4))
    
    # 可视化分类边界
    for clf, lab, grd, in zip([clf1, clf2, clf3, eclf],
                              ['Logistic Regression', 'Random Forest', 'RBF kernel SVM', 'Ensemble'],
                              itertools.product([0, 1], repeat=2)):
        clf.fit(X, y)
        ax = plt.subplot(gs[0, grd[0] * 2 + grd[1]])
        fig = plot_decision_regions(X=X, y=y, clf=clf, legend=2)
        plt.title(lab)
    plt.show()

## averaging

## ranking

## 3.1 blending

## 3.4 stacking

## 3.2 bagging

### 3.2.1 Bagging算法  
Bagging算法，又称装袋算法，是机器学习领域的一种集成学习算法。最初由Leo Breiman于1994年提出。之所以被称为装袋法，是因为它采用了一种有放回的抽样方法来生成训练数据。通过多轮有放回的对初始训练集进行随机采样，多个训练集被并行化生成，对应可训练出多个基学习器（基学习器间不存在强依赖关系），再将这些基学习器结合，构建出强学习器。其本质是引入了样本扰动，通过增加样本随机性，达到降低方差的效果。

<br></br>
<center><img src="./img/9210113-8ae1f9ba634caf4f.png" width="600" hegiht="" ></center>
<center><br>bagging流程图</br></center>
<br></br>

因为 bagging 方法可以减小过拟合，所以通常在强分类器和复杂模型上使用时表现的很好（例如，完全生长的决策树，fully developed decision trees），相比之下 boosting 方法则在弱模型上表现更好（例如，浅层决策树，shallow decision trees）。


### 3.2.2 执行步骤
假设有1000个样本，70%的样本作为训练集，30%的样本作为测试集。
- STEP1：从训练集700条里样本随机抽出500条样本，用来训练，于是生成了一个基学习器。
- STEP2：然后有放回的再从700条样本在随机抽出500条样本，用来训练，于是又生成了一个基学习器
- STEP3：假设以步骤2的方式随机抽取100次，这样就生成了100个基学习器了
- STEP4：用这100个基学习器对测试集的每一个样本都进行测试，然后通过投票的方式决定测集样本的预测结果。如果一个样本被这100个基学习器投票，即预测分类，被分为1的票数有80票，被分为0的有20票，显然，这个样本的预测结果为1。


### 3.2.3 使用sklearn库实现Bagging方法
**BaggingClassifier**  
决策树是一个可读性很强、分类快，同时不需要对数据进行归一化还有缩放的处理。但是决策树有个不足的地方就是非常容易过拟合，所以必须要采取一些方法进行剪枝。而bagging方法的出现，可以完美地解决了决策树过拟合的问题，同时bagging的使用也会使分类器分类效果得到了显著的提高。  
在 scikit-learn 中，bagging 方法使用统一的 BaggingClassifier 元估计器.

**导入相关包**

In [ ]:
#-*- coding:utf-8 -*-

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

**载入iris数据集**

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

# print('feature=',X)
# print('target=',y)

**bagging融合**

In [ ]:
def BaggingMethod(X, y):
    '''
    Bagging方法实现分类
    INPUT -> 特征, 分类标签
    '''
    scaler = StandardScaler() # 标准化转换
    scaler.fit(X)  # 训练标准化对象
    traffic_feature= scaler.transform(X)   # 转换数据集
    feature_train, feature_test, target_train, target_test = train_test_split(X, y, test_size=0.3, random_state=0)
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=None)

    clf = BaggingClassifier(base_estimator=tree, 
                            n_estimators=500,   # 生成500个决策树
                            max_samples=1.0,
                            max_features=1.0,
                            bootstrap=True,
                            bootstrap_features=False,
                            n_jobs=1,
                            random_state=1)
    clf.fit(feature_train, target_train)

    # 模型测试
    predict_results = clf.predict(feature_test)
    print(accuracy_score(predict_results, target_test))
    conf_mat = confusion_matrix(target_test, predict_results)
    print(conf_mat)
    print(classification_report(target_test, predict_results))

    return clf

**主程序**

------

**参数说明：**

1. base_estimator：基本模型，None默认为决策树；
2. n_estimators=500：生成500个决策；
3. max_samples : int或float，可选（默认值= 1.0），从X抽取以训练每个基本模型（estimator）的样本数，如果为int，则抽取样本 max_samples，如果float，则抽取本 max_samples * X.shape[0]。
4. max_features：int或float，可选（默认值= 1.0）从X绘制以训练每个基本估计量的要素数量。如果为int，则绘制特征 max_features。如果是浮动的，则绘制特征 max_features * X.shape[1]。
5. bootstrap : 布尔值，可选（默认= True）是否抽取样本进行替换。如果为False，则执行不替换的采样。
6. n_jobs : int或None（可选）（默认为None）fit和 并行运行的作业数predict。None除非joblib.parallel_backend上下文中，否则表示1 。-1表示使用所有处理器。
7. random_state : 整型，RandomState实例或无，可选（默认值：无）如果为int，则random_state是随机数生成器使用的种子；否则为false。

------

In [ ]:
if __name__ == '__main__':
    model = BaggingMethod(X, y)

### 3.2.4 参考资料
1. 机器学习中的集成方法（2）--Bagging（装袋法）（https://www.jianshu.com/p/fa212d4391f7）;
2. scikit-learn中BaggingClassifier参数（https://blog.csdn.net/Asdas_/article/details/104055529）；

## 3.3 adaboost

# 4 模型验证

In [ ]:
## 4.1 